In [6]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D, Dropout
import pandas as pd
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split
import torch
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential, Model
from tqdm import tqdm
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import TopKCategoricalAccuracy

In [10]:
#Load train images

# train_datagen = image.ImageDataGenerator(rescale=1./255,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     validation_split=0.2) # set validation split

# train_generator = train_datagen.flow_from_directory(
#     '../input/car-brand-logos/Car_Brand_Logos/Train',
#     target_size=(224, 224),
#     batch_size=20,
#     class_mode='categorical',
#     subset='training') # set as training data

# validation_generator = train_datagen.flow_from_directory(
#     '../input/car-brand-logos/Car_Brand_Logos/Train', # same directory as training data
#     target_size=(224, 224),
#     batch_size=20,
#     class_mode='categorical',
#     subset='validation',
#     shuffle=False) # set as validation data

In [11]:
from keras.applications.vgg16 import VGG16

model = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))
x = Flatten()(model.output)
x = Dense(100, activation='relu')(x)
x = Dense(100, activation='relu')(x)
x = Dropout(0.05)(x)
x = Dense(100, activation='relu')(x)
x = Dropout(0.05)(x)
predictions = Dense(8, activation='softmax')(x) 

full_model = Model(inputs=model.input, outputs=predictions)
for layer in model.layers:
    layer.trainable = False

In [9]:
#For training

# full_model.compile(loss='categorical_crossentropy',
#                   optimizer=Adam(learning_rate=0.001),
#                   metrics=['acc', TopKCategoricalAccuracy(k=3)])
# history = full_model.fit_generator(
#     train_generator, 
#     validation_data = validation_generator,
#     workers=10,
#     epochs=10
# )

In [12]:
#save model

# model_json = full_model.to_json()
# with open("model.json", "w") as json_file:
#     json_file.write(model_json)
# full_model.save_weights("model.h5")

In [13]:
def process_image(image):
    image = cv2.resize(image, (224, 224), interpolation = cv2.INTER_AREA)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

In [16]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.densenet import preprocess_input
from keras.models import model_from_json

json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

labels = ["hyundai", "lexus", "mazda", "mercedes", "opes", "skoda", "toyata", "volkswagen"]

def PredictImage(imgPath):
    global prediction
    global top3
    image = load_img(imgPath, target_size=(224, 224))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)

    results = loaded_model.predict(image)
    pred = np.argmax(results)
    prediction = labels[pred]
    #print(prediction)
    top_values_index = sorted(range(len(results[0])), key=lambda i: results[0][i])[-3:]
    top3ind = top_values_index[::-1]
    top3 = [labels[top3ind[0]], labels[top3ind[1]], labels[top3ind[2]]]
    #print(top3)
    return prediction, top3

Loaded model from disk


In [23]:
import cv2

cap = cv2.VideoCapture(-1)
scale = 3
font = cv2.FONT_HERSHEY_PLAIN

ret, frame = cap.read()
height, width, channels = frame.shape

while True:
    ret, frame = cap.read()
    image = process_image(frame)
    print(model.predict(image))
        
        

    cv2.imshow("easy vision", frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

AttributeError: 'NoneType' object has no attribute 'shape'